# Appending, Merging, and Processing Pandas Dataframes
### Processing tank risk and household data for GPU visualizations

### Import statements

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Reading InfoUSA data

This is a pre-processed file including tank and household distances, transformed latitude longitude coordinates, as well as all of national risk index data for six natural hazards (earthquake, strong winds, hurricanes, tornadoes, cold floods, and riverine floods), as stipulated by our researcher. To understand how it was processed in more detail, visit processing notebok **05_all_us_dist_processing**.  We then drop columns we will not be using in our visualizations.

In [2]:
DATA_DIR = os.getcwd()
DATA_DIR = DATA_DIR.replace('processing', 'data')
DATA_DIR

'/hpc/home/at341/ondemand/codeplus-celine-dcc-package/data'

In [4]:
df_hh = pd.read_parquet(DATA_DIR + '/distances_all_hh.parquet')
df_hh.head()

,zip,county_fips,state,child_num,age_code,lat_h_3857,lon_h_3857,lat_h_4326,lon_h_4326,erqk_risks,swnd_risks,hrcn_risks,trnd_risks,cfld_risks,rfld_risks,avg_risk,distance_m,distance_mi,distance_category
0,84606,37041,NH,12,H,-1.372560e+07,6.030085e+06,47.536477,-123.299151,23.971695,6.226360,-1.000000,11.394691,21.238291,6.071374,11.483735,7.411957e+04,46.055765,4
1,66723,23407,OR,3,A,-1.374498e+07,5.330436e+06,43.121187,-123.473248,2.593052,7.262484,-1.000000,9.914697,-1.000000,7.868933,4.606528,1.201416e+06,746.525231,4
2,59965,50536,NV,7,I,-1.235813e+07,4.645040e+06,38.462998,-111.014933,0.407726,8.066726,-1.000000,9.625419,-1.000000,6.357442,4.076219,1.334492e+06,829.214976,4
3,38676,38340,SD,3,B,-8.951334e+06,6.179301e+06,48.433631,-80.411200,3.312025,2.867939,-1.000000,10.280441,-1.000000,6.010181,3.745098,2.093508e+06,1300.845825,4
4,75640,24383,OR,15,J,-8.215682e+06,5.191497e+06,42.203405,-73.802723,3.862760,6.794995,3.156417,4.080984,-1.000000,5.920526,3.969280,2.549651e+05,158.427951,4


In [6]:
df_hh = df_hh.drop(['zip', 'county_fips', 'state', 'child_num', 'lat_h_4326','lon_h_4326', 'distance_m'], axis = 1)

### Using numpy's ```.where()``` to classify age codes

For our visualizations, we would like to have a tool allowing the user to select only points with elderly head of household (defining elderly as 65+, as used by the World Health Organization). Therefore, using the data dictionary provided by InfoUSA, age codes ```J```, ```K```, ```L```, or ```M``` will be classified as elderly.

Since we are processing this data to be used by Datashader through the Cuxfilter library, we need to be aware of some of the formatting of this dataframe so that it meets certain requirements. The Datashader plotting library that Cuxfilter uses to create our visualization through the use of Graphical Processing Units (GPUs) is optimized for working with large dataframes. This comes with a couple constraints, however. One of these is that Datashader only takes numerical inputs when creating the custom charts the user can interact with, like the multiselect chart or the range slider. This means that instead of being able to categorize each household by whether or not its head of household is eldery by labelling it with ```strings``` as ```'Elderly'``` or ```'No elderly'```, we must label it numerically. Therefore, we must convert each age code to a number that indicates whether or not that household has an elderly head of household.

This is done with the numpy library's ```.where()``` function, which uses if-else conditions to assign values in a new column. In the code below, if the age_code is ```J```, ```K```, ```L``` or ```M```, the household is marked as ```1```, meaning elderly, and marked as ```2```, not elderly, for all other values. 

In [7]:
df_hh['is_elderly'] = np.where((df_hh['age_code'] == 'J') | (df_hh['age_code'] == 'K') | (df_hh['age_code'] == 'L') | (df_hh['age_code'] == 'M'), 1, 2)
df_hh

,age_code,lat_h_3857,lon_h_3857,erqk_risks,swnd_risks,hrcn_risks,trnd_risks,cfld_risks,rfld_risks,avg_risk,distance_mi,distance_category,is_elderly
0,H,-1.372560e+07,6.030085e+06,23.971695,6.226360,-1.000000,11.394691,21.238291,6.071374,11.483735,46.055765,4,2
1,A,-1.374498e+07,5.330436e+06,2.593052,7.262484,-1.000000,9.914697,-1.000000,7.868933,4.606528,746.525231,4,2
2,I,-1.235813e+07,4.645040e+06,0.407726,8.066726,-1.000000,9.625419,-1.000000,6.357442,4.076219,829.214976,4,2
3,B,-8.951334e+06,6.179301e+06,3.312025,2.867939,-1.000000,10.280441,-1.000000,6.010181,3.745098,1300.845825,4,2
4,J,-8.215682e+06,5.191497e+06,3.862760,6.794995,3.156417,4.080984,-1.000000,5.920526,3.969280,158.427951,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
960402,M,-7.934295e+06,5.057504e+06,6.394724,4.414434,-1.000000,5.280541,-1.000000,41.749099,9.639800,2035.221681,4,1
960403,B,-1.010380e+07,4.934076e+06,2.169482,24.438503,-1.000000,37.424939,-1.000000,12.827024,12.809991,320.784963,4,2
960404,I,-1.089552e+07,4.319122e+06,5.315547,15.156209,20.793640,27.745481,2.048231,19.620172,15.113213,1069.111893,4,2
960405,J,-1.188947e+07,4.576454e+06,0.937095,14.637362,-1.000000,11.956780,-1.000000,4.829955,5.393532,1000.544765,4,1


### Using ```.rename()``` to rename columns

In addition, the Cuxfilter library only pulls coordinates from two columns: on latitude and one longitude column. This means that all the points displayed in the dashboard must be in the same column. Therefore, to plot tanks and households on the same dashboard, we append the dataframe with the coordinates for each tank to the dataframe with the coordinates for each household. To do so, the columns must be the same across both columns. Therefore, we renamed the ```lat_h_3857``` and ```lon_h_3857``` columns in the ```df_hh``` dataframe to ```lat_3857``` and ```lon_3857```. When the ```df_tanks``` dataframe is appended to this one, we will have general latitude and longitude columns including coordinate information for all the households and tanks in the US.

In [8]:
df_hh.rename(columns = {'lat_h_3857': 'lat_3857'}, inplace = True)
df_hh.rename(columns = {'lon_h_3857': 'lon_3857'}, inplace = True)
df_hh

,age_code,lat_3857,lon_3857,erqk_risks,swnd_risks,hrcn_risks,trnd_risks,cfld_risks,rfld_risks,avg_risk,distance_mi,distance_category,is_elderly
0,H,-1.372560e+07,6.030085e+06,23.971695,6.226360,-1.000000,11.394691,21.238291,6.071374,11.483735,46.055765,4,2
1,A,-1.374498e+07,5.330436e+06,2.593052,7.262484,-1.000000,9.914697,-1.000000,7.868933,4.606528,746.525231,4,2
2,I,-1.235813e+07,4.645040e+06,0.407726,8.066726,-1.000000,9.625419,-1.000000,6.357442,4.076219,829.214976,4,2
3,B,-8.951334e+06,6.179301e+06,3.312025,2.867939,-1.000000,10.280441,-1.000000,6.010181,3.745098,1300.845825,4,2
4,J,-8.215682e+06,5.191497e+06,3.862760,6.794995,3.156417,4.080984,-1.000000,5.920526,3.969280,158.427951,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
960402,M,-7.934295e+06,5.057504e+06,6.394724,4.414434,-1.000000,5.280541,-1.000000,41.749099,9.639800,2035.221681,4,1
960403,B,-1.010380e+07,4.934076e+06,2.169482,24.438503,-1.000000,37.424939,-1.000000,12.827024,12.809991,320.784963,4,2
960404,I,-1.089552e+07,4.319122e+06,5.315547,15.156209,20.793640,27.745481,2.048231,19.620172,15.113213,1069.111893,4,2
960405,J,-1.188947e+07,4.576454e+06,0.937095,14.637362,-1.000000,11.956780,-1.000000,4.829955,5.393532,1000.544765,4,1


### Reading and Processing Tank Data

This is anothe pre-processed dataframe, which contains the coordinates of each tank and also each of the six national risk index values associated with each tank based on county.  To understand how it was processed in more detail, visit processing notebok **04_risk_per_tank**. We then drop columns we will not need in our visualizations, and rename columns as explained above.

In [11]:
df_tanks = gpd.read_file(DATA_DIR + '/tanks_risk_score.shp')
df_tanks = df_tanks.drop(['state', 'tank_type', 'diameter', 'county', 'on_floodpl'], axis = 1)
df_tanks.head(n=3)

,lat_t_4326,lon_t_4326,lat_t_3857,lon_t_3857,erqk_risks,swnd_risks,hrcn_risks,trnd_risks,cfld_risks,rfld_risks,avg_risk,adj_risk,geometry
0,30.501991,-91.188296,-1.015103e+07,3.568241e+06,4.149297,9.661013,14.415955,43.776313,9.471153,39.822684,20.216069,20.216069,POINT (-91.18830 30.50199)
1,29.990189,-90.395876,-1.006282e+07,3.502289e+06,1.208395,6.264728,13.189863,13.190995,17.685820,12.877608,10.736235,10.736235,POINT (-90.39588 29.99019)
2,34.221754,-83.783722,-9.326761e+06,4.058617e+06,5.628088,12.104342,5.312985,31.912282,-1.000000,7.696209,10.442318,10.442318,POINT (-83.78372 34.22175)


In [12]:
df_tanks.rename(columns = {'lat_t_3857': 'lat_3857'}, inplace = True)
df_tanks.rename(columns = {'lon_t_3857': 'lon_3857'}, inplace = True)

In order for the tanks to display on Cuxfilter when using the distance range slider, we set the distance to the maximum distance between a household and a tank. This is because the distance column in the final merged dataframe used in our visualizations will represent the distance between a household and the tank nearest to it. However, for tanks, there is no associated distance, and when users play with the distance range slider, tanks will not appear on the visualization. We get around this by setting the distance to the maximum distance between a household and a tank. This is a limited solution potentially solveable by calculating the distance for each tank to the nearest household and including those values.

We add the ```distance_category``` and ```is_elderly``` columns to the ```df_tanks_harris``` dataframe, setting all their values to ```0``` to indicate that the point is a tank when plotted on the dashboard.

In [13]:
print(df_hh['distance_mi'].max())

3136.7234831188835


In [14]:
df_tanks['distance_mi'] = 3200
df_tanks['distance_category'] = 0
df_tanks['is_elderly'] = 0
df_tanks.head(n=3)

,lat_t_4326,lon_t_4326,lat_3857,lon_3857,erqk_risks,swnd_risks,hrcn_risks,trnd_risks,cfld_risks,rfld_risks,avg_risk,adj_risk,geometry,distance_mi,distance_category,is_elderly
0,30.501991,-91.188296,-1.015103e+07,3.568241e+06,4.149297,9.661013,14.415955,43.776313,9.471153,39.822684,20.216069,20.216069,POINT (-91.18830 30.50199),3200,0,0
1,29.990189,-90.395876,-1.006282e+07,3.502289e+06,1.208395,6.264728,13.189863,13.190995,17.685820,12.877608,10.736235,10.736235,POINT (-90.39588 29.99019),3200,0,0
2,34.221754,-83.783722,-9.326761e+06,4.058617e+06,5.628088,12.104342,5.312985,31.912282,-1.000000,7.696209,10.442318,10.442318,POINT (-83.78372 34.22175),3200,0,0


### Using ```.append()``` to join the two dataframes

Here we are appending the ```df_tanks```, our tank data, to ```df_hh```, our household data. Once this data is appended, the```.drop()``` method to drop columns we will not use in our visualizations. In this method, ```axis``` is set to ```1``` to indicate that we are dropping columns, not indices.

We then save this as a ```.parquet()``` file.

In [15]:
df = df_hh.append(df_tanks, ignore_index=True)
df = df.drop(['lat_t_4326', 'lon_t_4326', 'adj_risk', 'geometry'], axis = 1)
df

/tmp/ipykernel_1405583/423009934.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_hh.append(df_tanks, ignore_index=True)


,age_code,lat_3857,lon_3857,erqk_risks,swnd_risks,hrcn_risks,trnd_risks,cfld_risks,rfld_risks,avg_risk,distance_mi,distance_category,is_elderly
0,H,-1.372560e+07,6.030085e+06,23.971695,6.226360,-1.000000,11.394691,21.238291,6.071374,11.483735,46.055765,4,2
1,A,-1.374498e+07,5.330436e+06,2.593052,7.262484,-1.000000,9.914697,-1.000000,7.868933,4.606528,746.525231,4,2
2,I,-1.235813e+07,4.645040e+06,0.407726,8.066726,-1.000000,9.625419,-1.000000,6.357442,4.076219,829.214976,4,2
3,B,-8.951334e+06,6.179301e+06,3.312025,2.867939,-1.000000,10.280441,-1.000000,6.010181,3.745098,1300.845825,4,2
4,J,-8.215682e+06,5.191497e+06,3.862760,6.794995,3.156417,4.080984,-1.000000,5.920526,3.969280,158.427951,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
961384,NaN,-1.010035e+07,5.222881e+06,1.575536,17.648163,4.544047,21.537919,-1.000000,12.580429,9.647682,3200.000000,0,0
961385,NaN,-1.183249e+07,5.291041e+06,3.312025,2.867939,-1.000000,10.280441,-1.000000,6.010181,3.745098,3200.000000,0,0
961386,NaN,-9.971313e+06,4.384699e+06,17.807754,23.810359,8.253384,24.042775,-1.000000,18.432187,15.391077,3200.000000,0,0
961387,NaN,-7.944992e+06,5.135812e+06,9.400549,11.049468,5.819224,19.608082,7.130619,21.502062,12.418334,3200.000000,0,0


In [16]:
df.to_parquet(DATA_DIR + '/hh_and_tank_risks.parquet')

### Processing dataframes for individual natural hazards

Since we want to plot all of the risk dashboards separately, are now breaking down this dataframe, one for each type of natural hazard.

#### Processing the dataframe for earthquake risk

We are taking the macro dataframe we made above and dropping all of the other risks except for earthquake before exporting it to a parquet file to be used in visualizations.

In [17]:
df_erqk = df.drop(['swnd_risks', 'hrcn_risks', 'trnd_risks', 'cfld_risks', 'rfld_risks', 'avg_risk'], axis = 1)
df_erqk = df_erqk[['is_elderly', 'distance_mi', 'erqk_risks', 'lat_3857', 'lon_3857']]
df_erqk.rename(columns = {'erqk_risks': 'earthquake_risk'}, inplace = True)
df_erqk

,is_elderly,distance_mi,earthquake_risk,lat_3857,lon_3857
0,2,46.055765,23.971695,-1.372560e+07,6.030085e+06
1,2,746.525231,2.593052,-1.374498e+07,5.330436e+06
2,2,829.214976,0.407726,-1.235813e+07,4.645040e+06
3,2,1300.845825,3.312025,-8.951334e+06,6.179301e+06
4,1,158.427951,3.862760,-8.215682e+06,5.191497e+06
...,...,...,...,...,...
961384,0,3200.000000,1.575536,-1.010035e+07,5.222881e+06
961385,0,3200.000000,3.312025,-1.183249e+07,5.291041e+06
961386,0,3200.000000,17.807754,-9.971313e+06,4.384699e+06
961387,0,3200.000000,9.400549,-7.944992e+06,5.135812e+06


In [28]:
df_erqk.to_parquet(DATA_DIR + '/earthquake_risk.parquet')

#### Processing dataframe for strong wind risk
Following the same steps as above, this time dropping all of the other risks except for strong wind risk before exporting it to a parquet file to be used in visualizations.

In [19]:
df_swnd = df.drop(['erqk_risks', 'hrcn_risks', 'trnd_risks', 'cfld_risks', 'rfld_risks', 'avg_risk'], axis = 1)
df_swnd = df_swnd[['is_elderly', 'distance_mi', 'swnd_risks', 'lat_3857', 'lon_3857']]
df_swnd.rename(columns = {'swnd_risks': 'strong_wind_risk'}, inplace = True)
df_swnd

,is_elderly,distance_mi,strong_wind_risk,lat_3857,lon_3857
0,2,46.055765,6.226360,-1.372560e+07,6.030085e+06
1,2,746.525231,7.262484,-1.374498e+07,5.330436e+06
2,2,829.214976,8.066726,-1.235813e+07,4.645040e+06
3,2,1300.845825,2.867939,-8.951334e+06,6.179301e+06
4,1,158.427951,6.794995,-8.215682e+06,5.191497e+06
...,...,...,...,...,...
961384,0,3200.000000,17.648163,-1.010035e+07,5.222881e+06
961385,0,3200.000000,2.867939,-1.183249e+07,5.291041e+06
961386,0,3200.000000,23.810359,-9.971313e+06,4.384699e+06
961387,0,3200.000000,11.049468,-7.944992e+06,5.135812e+06


In [20]:
df_swnd.to_parquet(DATA_DIR + '/strong_wind_risk.parquet')

#### Processing dataframe for hurricane risk
Following the same steps as above, this time dropping all of the other risks except for hurricane risk before exporting it to a parquet file to be used in visualizations.

In [21]:
df_hrcn = df.drop(['erqk_risks','swnd_risks', 'trnd_risks', 'cfld_risks', 'rfld_risks', 'avg_risk'], axis = 1)
df_hrcn = df_hrcn[['is_elderly', 'distance_mi', 'hrcn_risks', 'lat_3857', 'lon_3857']]
df_hrcn.rename(columns = {'hrcn_risks': 'hurricane_risk'}, inplace = True)
df_hrcn

,is_elderly,distance_mi,hurricane_risk,lat_3857,lon_3857
0,2,46.055765,-1.000000,-1.372560e+07,6.030085e+06
1,2,746.525231,-1.000000,-1.374498e+07,5.330436e+06
2,2,829.214976,-1.000000,-1.235813e+07,4.645040e+06
3,2,1300.845825,-1.000000,-8.951334e+06,6.179301e+06
4,1,158.427951,3.156417,-8.215682e+06,5.191497e+06
...,...,...,...,...,...
961384,0,3200.000000,4.544047,-1.010035e+07,5.222881e+06
961385,0,3200.000000,-1.000000,-1.183249e+07,5.291041e+06
961386,0,3200.000000,8.253384,-9.971313e+06,4.384699e+06
961387,0,3200.000000,5.819224,-7.944992e+06,5.135812e+06


In [22]:
df_hrcn.to_parquet(DATA_DIR + '/hurricane_risk.parquet')

#### Processing dataframe for tornado risk
Following the same steps as above, this time dropping all of the other risks except for tornado risk before exporting it to a parquet file to be used in visualizations.

In [23]:
df_trnd = df.drop(['erqk_risks','swnd_risks', 'hrcn_risks', 'cfld_risks', 'rfld_risks', 'avg_risk'], axis = 1)
df_trnd = df_trnd[['is_elderly', 'distance_mi','trnd_risks', 'lat_3857', 'lon_3857']]
df_trnd.rename(columns = {'trnd_risks': 'tornado_risk'}, inplace = True)
df_trnd

,is_elderly,distance_mi,tornado_risk,lat_3857,lon_3857
0,2,46.055765,11.394691,-1.372560e+07,6.030085e+06
1,2,746.525231,9.914697,-1.374498e+07,5.330436e+06
2,2,829.214976,9.625419,-1.235813e+07,4.645040e+06
3,2,1300.845825,10.280441,-8.951334e+06,6.179301e+06
4,1,158.427951,4.080984,-8.215682e+06,5.191497e+06
...,...,...,...,...,...
961384,0,3200.000000,21.537919,-1.010035e+07,5.222881e+06
961385,0,3200.000000,10.280441,-1.183249e+07,5.291041e+06
961386,0,3200.000000,24.042775,-9.971313e+06,4.384699e+06
961387,0,3200.000000,19.608082,-7.944992e+06,5.135812e+06


In [24]:
df_trnd.to_parquet(DATA_DIR + '/tornado_risk.parquet')

#### Processing dataframe for coastal flood risk
Following the same steps as above, this time dropping all of the other risks except for coastal flood risk before exporting it to a parquet file to be used in visualizations.

In [25]:
df_cfld = df.drop(['erqk_risks','swnd_risks', 'trnd_risks', 'hrcn_risks', 'rfld_risks', 'avg_risk'], axis = 1)
df_cfld = df_cfld[['is_elderly', 'distance_mi','cfld_risks', 'lat_3857', 'lon_3857']]
df_cfld.rename(columns = {'cfld_risks': 'coastal_flood_risk'}, inplace = True)
df_cfld

,is_elderly,distance_mi,coastal_flood_risk,lat_3857,lon_3857
0,2,46.055765,21.238291,-1.372560e+07,6.030085e+06
1,2,746.525231,-1.000000,-1.374498e+07,5.330436e+06
2,2,829.214976,-1.000000,-1.235813e+07,4.645040e+06
3,2,1300.845825,-1.000000,-8.951334e+06,6.179301e+06
4,1,158.427951,-1.000000,-8.215682e+06,5.191497e+06
...,...,...,...,...,...
961384,0,3200.000000,-1.000000,-1.010035e+07,5.222881e+06
961385,0,3200.000000,-1.000000,-1.183249e+07,5.291041e+06
961386,0,3200.000000,-1.000000,-9.971313e+06,4.384699e+06
961387,0,3200.000000,7.130619,-7.944992e+06,5.135812e+06


In [29]:
df_cfld.to_parquet(DATA_DIR + '/coastal_flood_risk.parquet')

#### Processing dataframe for riverine flood risk
Following the same steps as above, this time dropping all of the other risks except for riverine flood risk before exporting it to a parquet file to be used in visualizations.

In [30]:
df_rfld = df.drop(['erqk_risks','swnd_risks', 'trnd_risks', 'hrcn_risks', 'cfld_risks', 'avg_risk'], axis = 1)
df_rfld = df_rfld[[ 'is_elderly', 'distance_mi', 'rfld_risks', 'lat_3857', 'lon_3857']]
df_rfld.rename(columns = {'rfld_risks': 'riverine_flood_risk'}, inplace = True)
df_rfld

,is_elderly,distance_mi,riverine_flood_risk,lat_3857,lon_3857
0,2,46.055765,6.071374,-1.372560e+07,6.030085e+06
1,2,746.525231,7.868933,-1.374498e+07,5.330436e+06
2,2,829.214976,6.357442,-1.235813e+07,4.645040e+06
3,2,1300.845825,6.010181,-8.951334e+06,6.179301e+06
4,1,158.427951,5.920526,-8.215682e+06,5.191497e+06
...,...,...,...,...,...
961384,0,3200.000000,12.580429,-1.010035e+07,5.222881e+06
961385,0,3200.000000,6.010181,-1.183249e+07,5.291041e+06
961386,0,3200.000000,18.432187,-9.971313e+06,4.384699e+06
961387,0,3200.000000,21.502062,-7.944992e+06,5.135812e+06


In [31]:
df_rfld.to_parquet(DATA_DIR + '/riverine_flood_risk.parquet')

#### Processing dataframe for average risk
Following the same steps as above, this time dropping all of the other risks except for average risk before exporting it to a parquet file to be used in visualizations.

In [32]:
df_avg = df.drop(['erqk_risks','swnd_risks', 'trnd_risks', 'hrcn_risks', 'cfld_risks', 'rfld_risks'], axis = 1)
df_avg = df_avg[[ 'is_elderly', 'distance_mi', 'avg_risk', 'lat_3857', 'lon_3857']]
df_avg.rename(columns = {'avg_risk': 'average_risk'}, inplace = True)
df_avg

,is_elderly,distance_mi,average_risk,lat_3857,lon_3857
0,2,46.055765,11.483735,-1.372560e+07,6.030085e+06
1,2,746.525231,4.606528,-1.374498e+07,5.330436e+06
2,2,829.214976,4.076219,-1.235813e+07,4.645040e+06
3,2,1300.845825,3.745098,-8.951334e+06,6.179301e+06
4,1,158.427951,3.969280,-8.215682e+06,5.191497e+06
...,...,...,...,...,...
961384,0,3200.000000,9.647682,-1.010035e+07,5.222881e+06
961385,0,3200.000000,3.745098,-1.183249e+07,5.291041e+06
961386,0,3200.000000,15.391077,-9.971313e+06,4.384699e+06
961387,0,3200.000000,12.418334,-7.944992e+06,5.135812e+06


In [33]:
df_avg.to_parquet(DATA_DIR + '/avg_risk.parquet')